In [6]:
DATA_NAME = 'syntheticMRI2D-axial' 
TRANSFORM = 'gabor'
CHANNEL = ''
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [7]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gabor_axial_syntheticMRI2D.ipynb


In [8]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [9]:
from scipy import spatial
from sklearn.decomposition import PCA

In [10]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [11]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [12]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [13]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [14]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,20394.70,0.53,-0.07,0.15,0.02,-0.04,-0.02,-0.78,0.31,-0.22,...,-0.03,0.04,-0.00,0.49,0.04,-0.17,0.03,0.03,0.01,0.01
1,0.53,10011.01,0.40,-0.18,-0.03,-0.03,-0.01,0.66,-0.09,-0.25,...,-0.00,0.00,0.00,0.29,-0.06,-0.02,-0.08,0.00,0.01,0.01
2,-0.07,0.40,4311.39,-0.02,0.01,0.02,0.01,0.02,0.29,-0.16,...,0.01,-0.01,0.01,-0.34,0.18,0.00,-0.07,0.00,0.02,-0.00
3,0.15,-0.18,-0.02,1111.48,-0.01,0.01,-0.00,-0.25,-0.09,0.01,...,-0.01,-0.00,0.00,0.13,-0.03,0.03,0.01,-0.00,-0.00,-0.00
4,0.02,-0.03,0.01,-0.01,214.98,-0.00,0.00,0.02,-0.01,0.07,...,-0.00,0.00,0.00,0.03,0.02,-0.02,0.01,0.00,0.00,-0.00
5,-0.04,-0.03,0.02,0.01,-0.00,37.56,0.00,-0.05,0.02,-0.01,...,0.00,-0.00,0.00,-0.02,0.00,-0.00,0.01,0.00,0.00,-0.00
6,-0.02,-0.01,0.01,-0.00,0.00,0.00,18.26,0.01,0.00,0.02,...,0.00,-0.00,-0.00,-0.01,-0.00,0.00,-0.00,0.00,-0.00,0.00
7,-0.78,0.66,0.02,-0.25,0.02,-0.05,0.01,23523.21,-0.00,0.41,...,0.01,0.01,-0.01,0.16,-0.47,-0.06,-0.11,0.04,0.02,-0.02
8,0.31,-0.09,0.29,-0.09,-0.01,0.02,0.00,-0.00,11778.36,0.18,...,-0.02,0.01,0.00,0.31,0.18,-0.21,0.05,0.04,-0.00,-0.00


In [15]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00004,-0.00001,0.00003,0.00001,-0.00004,-0.00003,-0.00004,0.00002,-0.00002,...,-0.00003,0.00008,-0.00001,0.00003,0.00000,-0.00004,0.00001,0.00002,0.00001,0.00002
1,0.00004,1.00000,0.00006,-0.00005,-0.00002,-0.00005,-0.00001,0.00004,-0.00001,-0.00003,...,-0.00000,0.00000,0.00001,0.00003,-0.00001,-0.00001,-0.00005,0.00000,0.00002,0.00004
2,-0.00001,0.00006,1.00000,-0.00001,0.00001,0.00005,0.00004,0.00000,0.00004,-0.00003,...,0.00002,-0.00003,0.00003,-0.00005,0.00004,0.00000,-0.00006,0.00000,0.00006,-0.00001
3,0.00003,-0.00005,-0.00001,1.00000,-0.00003,0.00003,-0.00002,-0.00005,-0.00002,0.00001,...,-0.00003,-0.00003,0.00002,0.00003,-0.00001,0.00003,0.00002,-0.00001,-0.00002,-0.00001
4,0.00001,-0.00002,0.00001,-0.00003,1.00000,-0.00002,0.00000,0.00001,-0.00001,0.00006,...,-0.00000,0.00003,0.00001,0.00002,0.00003,-0.00003,0.00005,0.00002,0.00003,-0.00004
5,-0.00004,-0.00005,0.00005,0.00003,-0.00002,1.00000,0.00000,-0.00006,0.00003,-0.00002,...,0.00001,-0.00004,0.00004,-0.00003,0.00000,-0.00001,0.00005,0.00001,0.00004,-0.00001
6,-0.00003,-0.00001,0.00004,-0.00002,0.00000,0.00000,1.00000,0.00002,0.00000,0.00005,...,0.00004,-0.00001,-0.00003,-0.00003,-0.00000,0.00002,-0.00001,0.00004,-0.00002,0.00002
7,-0.00004,0.00004,0.00000,-0.00005,0.00001,-0.00006,0.00002,1.00000,-0.00000,0.00004,...,0.00001,0.00002,-0.00003,0.00001,-0.00005,-0.00001,-0.00004,0.00003,0.00003,-0.00005
8,0.00002,-0.00001,0.00004,-0.00002,-0.00001,0.00003,0.00000,-0.00000,1.00000,0.00002,...,-0.00003,0.00001,0.00000,0.00002,0.00003,-0.00006,0.00003,0.00005,-0.00001,-0.00001


In [16]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

4.694096558296175

In [17]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.32163797e+07 9.90827880e+06 5.50169286e+06 4.26695829e+06
 3.98837031e+06 3.62215497e+06 3.24572778e+06 2.36092265e+06
 7.97843294e+05 7.00554096e+05 6.57261995e+05 4.39470799e+05
 3.26556467e+05 3.02554082e+05 1.22445463e+05 1.14572040e+05
 8.70015929e+04 3.28827219e+04 2.96889952e+04 2.66699412e+04
 1.15268129e+04 8.00202265e+03 3.88919808e+03 2.30229178e+03
 2.02243408e+03 1.05764945e+03 4.78934444e+02 2.33044431e+02
 1.30224291e+02 9.45438320e+01 6.80692739e+01 3.19818723e+01
 2.09190802e+01 1.19718184e+01 1.02042627e+01 8.39887374e+00
 7.39314348e+00 5.05275664e+00 2.86812495e+00 2.41075626e+00
 2.08525600e+00 2.42129350e-24]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.078917,-0.010963,-0.004001,-0.001010,-0.000192,-0.000036,-1.569853e-05,0.995957,-0.014191,-0.005494,...,-0.000049,-0.000015,-0.000009,-0.016753,-0.003397,-0.000983,-0.000274,-0.000056,-0.000016,-9.414292e-06
1,0.994838,-0.015428,-0.005269,-0.001292,-0.000249,-0.000046,-2.218916e-05,0.075938,-0.020802,-0.007348,...,-0.000066,-0.000017,-0.000011,-0.025544,-0.004422,-0.001286,-0.000349,-0.000074,-0.000022,-1.065968e-05
2,0.032428,-0.023993,-0.006286,-0.001506,-0.000289,-0.000049,-2.563900e-05,0.021496,-0.040884,-0.009048,...,-0.000074,-0.000020,-0.000012,-0.066744,-0.005211,-0.001453,-0.000399,-0.000090,-0.000026,-1.102856e-05
3,0.015437,-0.024400,-0.005018,-0.001159,-0.000222,-0.000036,-1.981187e-05,0.011337,-0.059427,-0.007420,...,-0.000057,-0.000018,-0.000011,-0.303143,-0.004119,-0.001150,-0.000313,-0.000069,-0.000021,-9.881557e-06
4,0.021150,-0.041378,-0.007857,-0.001786,-0.000342,-0.000061,-3.123866e-05,0.015840,-0.124399,-0.011641,...,-0.000086,-0.000026,-0.000015,0.911190,-0.006418,-0.001769,-0.000492,-0.000102,-0.000031,-1.665670e-05
5,0.021991,-0.059637,-0.009560,-0.002158,-0.000416,-0.000074,-3.391409e-05,0.016674,-0.347062,-0.014438,...,-0.000109,-0.000034,-0.000017,0.199079,-0.007807,-0.002115,-0.000585,-0.000125,-0.000036,-1.753167e-05
6,0.031851,-0.134671,-0.016620,-0.003726,-0.000715,-0.000123,-6.047586e-05,0.024728,0.920799,-0.025572,...,-0.000186,-0.000056,-0.000030,0.167087,-0.013529,-0.003676,-0.000995,-0.000210,-0.000063,-3.157636e-05
7,0.021506,0.986335,-0.018522,-0.003943,-0.000752,-0.000134,-6.446682e-05,0.017328,0.094805,-0.030783,...,-0.000194,-0.000059,-0.000032,0.061805,-0.014857,-0.003859,-0.001056,-0.000224,-0.000065,-3.136607e-05
8,0.006027,0.016788,-0.034567,-0.003987,-0.000728,-0.000132,-6.062342e-05,0.005081,0.012514,0.975058,...,-0.000191,-0.000058,-0.000033,0.010836,-0.021590,-0.003903,-0.001043,-0.000222,-0.000067,-3.140775e-05
9,0.000030,-0.000023,-0.000354,-0.000024,0.000002,0.000005,-5.466347e-08,0.000031,0.000035,0.000448,...,0.000007,-0.000002,0.000002,0.000026,-0.000107,-0.000035,-0.000003,-0.000001,-0.000002,8.500657e-07


In [18]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.004043226582845505,
 0.005162096132710348,
 0.009110426755204348,
 0.05617342260134872,
 0.08881040176214439,
 0.09760110060964622,
 0.07920109956440058,
 0.013664785501938947,
 0.02494179226781479,
 0.29153773325199694,
 0.3157196923425426,
 0.01730355420791263,
 0.2389573832640982,
 0.25083895064809314,
 0.26232072504647586,
 0.27971634604330664,
 0.02368483041242897,
 0.05167752930691616,
 0.21250376590967857,
 0.23320066911903992,
 0.01396803497227117,
 0.014936740163257411,
 0.01189861564395489,
 0.13264142378307475,
 0.14187521915629275,
 0.0135434926638337,
 0.011180048151657296,
 0.015664590736305217,
 0.035704906888280985,
 0.0619786926484468,
 0.05659641579249042,
 0.03743250008976551,
 0.04861906464844301,
 0.10810815436917509,
 0.1704012548381595,
 0.24533436020236798,
 0.26392680994655915,
 0.41270131098392115,
 0.20175940187315045,
 0.34746859235383554,
 0.2413828832460102,
 0.5412652813080283]